<a href="https://colab.research.google.com/github/yukinaga/twitter_bot/blob/master/section5/02_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データの前処理
対話文のデータセットに前処理を行い、保存します。

## ライブラリのインストール
分かち書きのためにjanomeを、テキストデータの前処理のためにtorchtextをインストールします。

In [1]:
!pip install janome==0.4.1
!pip install -U torchtext==0.7.0

  Using cached Janome-0.4.1-py2.py3-none-any.whl (19.7 MB)


ERROR: Could not find a version that satisfies the requirement torchtext==0.7.0 (from versions: 0.1.1, 0.2.0, 0.2.1, 0.2.3, 0.3.1, 0.4.0, 0.5.0, 0.6.0)
ERROR: No matching distribution found for torchtext==0.7.0


## Google ドライブとの連携  
以下のコードを実行し、認証コードを使用してGoogle ドライブをマウントします。

In [3]:
!dir

 �h���C�u L �̃{�����[�� ���x���� WD25EZRX1 �ł�
 �{�����[�� �V���A���ԍ��� 1CAC-24E8 �ł�

 L:\Python�[�w�����w�K����\twitter_bot\section_5 �̃f�B���N�g��

2020/10/22  21:29    <DIR>          .
2020/10/22  21:29    <DIR>          ..
2020/10/22  21:26    <DIR>          .ipynb_checkpoints
2020/10/22  21:23             7,253 01_input_padding.ipynb
2020/10/22  21:29            14,440 02_preprocessing.ipynb
2020/10/19  20:50            21,597 03_attension.ipynb
               3 �̃t�@�C��              43,290 �o�C�g
               3 �̃f�B���N�g��  340,751,835,136 �o�C�g�̋󂫗̈�


## 対話文の取得
雑談対話コーパス「projectnextnlp-chat-dialogue-corpus.zip」をダウンロードします。  
  
> Copyright (c) 2015 Project Next NLP 対話タスク 参加者一同  
> https://sites.google.com/site/dialoguebreakdowndetection/chat-dialogue-corpus/LICENSE.txt  
> Released under the MIT license

解凍したフォルダをGoogle ドライブにアップします。  
フォルダからjsonファイルを読み込み、対話文として成り立っている文章を取り出してリストに格納します。  



In [7]:
import glob  # ファイルの取得に使用
import json  # jsonファイルの読み込みに使用
import re

path = "./projectnextnlp-chat-dialogue-corpus/json"  # フォルダの場所を指定

files = glob.glob(path + "/*/*.json")  # ファイルの一覧
dialogues = []  # 複数の対話文を格納するリスト
file_count= 0  # ファイル数のカウント
for file in files:
    with open(file, "r", encoding="utf-8_sig") as f:
        json_dic = json.load(f)
        dialogue = []  # 単一の対話
        for turn in json_dic["turns"]:
            annotations = turn["annotations"]  # 注釈
            speaker = turn["speaker"]  # 発言者
            utterance = turn["utterance"]  # 発言

            # 空の文章や、特殊文字や数字が含まれる文章は除く
            if (utterance=="") or ("\\u" in utterance) or (re.search("\d", utterance)!=None):
                dialogue.clear()  # 対話をリセット
                continue

            utterance = utterance.replace(".", "。").replace(",", "、")
            utterance = utterance.split("。")[0]

            if speaker=="U":  # 発言者が人間であれば
                dialogue.append(utterance) 
            else:  # 発言者がシステムであれば
                is_wrong = False
                for annotation in annotations:
                    breakdown = annotation["breakdown"]  # 分類
                    if breakdown=="X":  # 1つでも不適切評価があれば
                        is_wrong = True
                        break
                if is_wrong:
                    dialogue.clear()  # 対話をリセット
                else:
                    dialogue.append(utterance)  # 不適切評価が無ければ対話に追加
            
            if len(dialogue) >= 2:  # 単一の会話が成立すれば
                dialogues.append(dialogue.copy())
                dialogue.pop(0)  # 最初の要素を削除

    file_count += 1
    if file_count%100 == 0:
        print("files:", file_count, "dialogues", len(dialogues))

print("files:", file_count, "dialogues", len(dialogues))

files: 100 dialogues 666
files: 200 dialogues 2094
files: 300 dialogues 3488
files: 400 dialogues 4966
files: 500 dialogues 6447
files: 600 dialogues 7917
files: 700 dialogues 9418
files: 800 dialogues 10893
files: 900 dialogues 12357
files: 1000 dialogues 13801
files: 1100 dialogues 15244
files: 1146 dialogues 15903


## データ拡張の準備
データ拡張の準備として、正規表現の設定および分かち書きを行います。

In [8]:
import re
from janome.tokenizer import Tokenizer

re_kanji = re.compile(r"^[\u4E00-\u9FD0]+$")  # 漢字の検出用
re_katakana = re.compile(r"[\u30A1-\u30F4]+")  # カタカナの検出用
j_tk = Tokenizer()

def wakati(text):
    return [tok for tok in j_tk.tokenize(text, wakati=True)] 

wakati_inp = []  # 単語に分割された入力文
wakati_rep = []  # 単語に分割された応答文
for dialogue in dialogues:
    wakati_inp.append(wakati(dialogue[0])[:10])
    wakati_rep.append(wakati(dialogue[1])[:10])

## データ拡張
対話データの数を水増しします。  
ある入力文を、それに対応する応答文以外の複数の応答文と組み合わせます。  
組み合わせる応答文は、入力文に含まれる漢字やカタカナの単語を含むものを選択します。  

In [9]:
dialogues_plus = []
for i, w_inp in enumerate(wakati_inp):  # 全ての入力文でループ
    inp_count = 0  # ある入力から生成された対話文をカウント
    for j, w_rep in enumerate(wakati_rep):  # 全ての応答文でループ
        if i==j:
            dialogues_plus.append(["".join(w_inp), "".join(w_rep)])
            continue
        similarity = 0  # 類似度
        for w in w_inp:  # 入力文と同じ単語があり、それが漢字かカタカナであれば類似度を上げる
            if (w in w_rep) and (re_kanji.fullmatch(w) or re_katakana.fullmatch(w)):
                similarity += 1
        if similarity >= 1:
            dialogue_plus = ["".join(w_inp), "".join(w_rep)]
            if dialogue_plus not in dialogues_plus:
                dialogues_plus.append(dialogue_plus)
                inp_count += 1
                if inp_count >= 10:  # ある入力から生成する対話文の上限
                    break

    if i%1000 == 0:
        print("i:", i, "dialogues_pus:", len(dialogues_plus))

print("i:", i, "dialogues_pus:", len(dialogues_plus))

i: 0 dialogues_pus: 9
i: 1000 dialogues_pus: 5438
i: 2000 dialogues_pus: 10843
i: 3000 dialogues_pus: 16958
i: 4000 dialogues_pus: 22974
i: 5000 dialogues_pus: 28754
i: 6000 dialogues_pus: 34082
i: 7000 dialogues_pus: 39029
i: 8000 dialogues_pus: 44807
i: 9000 dialogues_pus: 50732
i: 10000 dialogues_pus: 56734
i: 11000 dialogues_pus: 62595
i: 12000 dialogues_pus: 68577
i: 13000 dialogues_pus: 74017
i: 14000 dialogues_pus: 79608
i: 15000 dialogues_pus: 85222
i: 15902 dialogues_pus: 90749


拡張された対話データを、新たな対話データとします。

In [10]:
dialogues = dialogues_plus

## 対話データの保存
対話データをcsvファイルとしてGoogle Driveに保存します。

In [12]:
import csv
from sklearn.model_selection import train_test_split

dialogues_train, dialogues_test =  train_test_split(dialogues, shuffle=True, test_size=0.05)  # 5%がテストデータ
path = "./"  # 保存場所

# utf-8で保存するように変更する必要あり！！！！

with open(path+"dialogues_train.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(dialogues_train)

with open(path+"dialogues_test.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(dialogues_test)

## 対話文の取得
Googleドライブから、対話文のデータを取り出してデータセットに格納します。  



In [16]:
import torch
import torchtext
from janome.tokenizer import Tokenizer

path = "./"  # 保存場所を指定

j_tk = Tokenizer()
def tokenizer(text): 
    return [tok for tok in j_tk.tokenize(text, wakati=True)]  # 内包表記
 
# データセットの列を定義
input_field = torchtext.data.Field(  # 入力文
    sequential=True,  # データ長さが可変かどうか
    tokenize=tokenizer,  # 前処理や単語分割などのための関数
    batch_first=True,  # バッチの次元を先頭に
    lower=True  # アルファベットを小文字に変換
    )

reply_field = torchtext.data.Field(  # 応答文
    sequential=True,  # データ長さが可変かどうか
    tokenize=tokenizer,  # 前処理や単語分割などのための関数
    init_token = "<sos>",  # 文章開始のトークン
    eos_token = "<eos>",  # 文章終了のトークン
    batch_first=True,  # バッチの次元を先頭に
    lower=True  # アルファベットを小文字に変換
    )
 
# csvファイルからデータセットを作成
train_data, test_data = torchtext.data.TabularDataset.splits(
    path=path,
    train="dialogues_train.csv",
    validation="dialogues_test.csv",
    format="csv",
    fields=[("inp_text", input_field), ("rep_text", reply_field)]  # 列の設定
    )

## 単語とインデックスの対応
単語にインデックスを割り振り、辞書として格納します。

In [17]:
input_field.build_vocab(
    train_data,
    min_freq=3,
    )
reply_field.build_vocab(
    train_data,
    min_freq=3,
    )

In [18]:
print(input_field.vocab.freqs)  # 各単語の出現頻度
print(len(input_field.vocab.stoi))
print(len(input_field.vocab.itos))
print(len(reply_field.vocab.stoi))
print(len(reply_field.vocab.itos))

Counter({'です': 44684, 'は': 34682, 'ね': 27513, 'が': 21355, 'か': 15555, '？': 13856, 'に': 13718, 'ます': 13236, 'の': 12659, 'を': 10773, 'よ': 9786, 'て': 9152, 'で': 7776, 'ん': 5837, 'いい': 5670, 'ない': 5661, '好き': 5233, 'も': 5184, 'た': 5103, '、': 4905, '海': 4881, 'と': 4746, 'な': 4684, 'し': 4352, '！': 4127, 'スイカ': 3967, 'だ': 3772, 'から': 3263, '行き': 2996, '退屈': 2884, 'たい': 2859, '気': 2703, '何': 2695, '症': 2467, '熱中': 2414, 'そう': 2365, '．': 2356, '私': 2316, 'う': 2111, 'つけ': 2102, 'ねー': 2080, 'こと': 1751, 'ねえ': 1717, 'てる': 1717, 'こんにちは': 1700, '夏': 1677, 'まし': 1636, 'あり': 1628, 'い': 1469, '人': 1419, '雨': 1393, '食べ': 1382, 'お': 1377, '見': 1377, 'ば': 1360, 'こんばんは': 1312, '良い': 1281, '，': 1270, '大丈夫': 1206, '最近': 1202, 'ませ': 1200, '楽しい': 1161, 'する': 1139, 'とか': 1082, '一緒': 1081, '多い': 1071, '朝': 1057, '大好き': 1046, 'けど': 1016, 'でしょ': 980, '今日': 975, '行っ': 941, '美味しい': 928, '僕': 907, 'いえ': 906, '仕事': 899, 'へ': 867, '心': 813, 'ましょ': 813, 'ある': 800, '趣味': 800, 'なり': 790, '天気': 775, '有名': 768, 'かも': 760, '大

## データセットの保存
データセットの`examples`とFieldをそれぞれ保存します。

In [20]:
import dill

torch.save(train_data.examples, path+"train_examples.pkl", pickle_module=dill)
torch.save(test_data.examples, path+"test_examples.pkl", pickle_module=dill)

torch.save(input_field, path+"input_field.pkl", pickle_module=dill)
torch.save(reply_field, path+"reply_field.pkl", pickle_module=dill)